In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Loading Dataset

In [2]:
import torch 
import torch.nn as nn
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
from torch.utils.data import DataLoader
train_transforms=v2.Compose([
    v2.ToImage(),
    v2.Resize((224,224)),
    v2.RandomRotation(10),
    v2.RandomHorizontalFlip(p=0.5),
    v2.GaussianBlur(kernel_size=(5,9),sigma=(0.1,5,)),
    v2.ToDtype(torch.float32,scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

In [3]:
animals_dataset=ImageFolder("/kaggle/input/zindis-animal-classification-challenge/Animal Classification Challenge/train",transform=train_transforms)

In [4]:
animals_dataset.classes[0]="elephant"
animals_dataset.classes[1]="zebra"

In [5]:
animals_dataset.classes

['elephant', 'zebra']

In [6]:
test_transforms=v2.Compose([
    v2.ToImage(),
    v2.Resize((224,224)),
    v2.ToDtype(torch.float32,scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    
])

In [7]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
labels=animals_dataset.targets
train_indices,val_indices=train_test_split(np.arange(len(labels)),test_size=0.1,random_state=42,stratify=labels)

In [8]:
animals_dataset.transform=test_transforms
val_dataset=Subset(animals_dataset,val_indices)
animals_dataset.transform=train_transforms
train_dataset=Subset(animals_dataset,train_indices)

In [9]:
from PIL import Image
from torch.utils.data import Dataset
class testDataset(Dataset):
    def __init__(self,img_dir,transform=None):
        self.img_dir=img_dir
        self.transform=transform
        self.img_names = sorted([f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))])
    def __len__(self):
        return len(self.img_names)
    def __getitem__(self,idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path
        

In [10]:
test_dataset=testDataset("/kaggle/input/zindis-animal-classification-challenge/Animal Classification Challenge/test",transform=test_transforms)

In [11]:
dataloader_train = DataLoader(train_dataset, shuffle=True, batch_size=32)
dataloader_val = DataLoader(val_dataset, shuffle=False, batch_size=32)
dataloader_test = DataLoader(test_dataset, shuffle=False, batch_size=32)


# Building Model

In [12]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [13]:
from torchvision.models import resnet18 ,ResNet18_Weights
model=resnet18(weights=ResNet18_Weights.IMAGENET1K_V1).to(device)


In [14]:
for param in model.parameters(): #freeze all base layers in the "features" section of the model 
    param.requires_grad = False

In [15]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
model.fc=nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=512,out_features=2,bias=True).to(device)
)

# Training The Model

In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau,StepLR
learning_rate=0.00001
epochs=10
loss_fn=nn.CrossEntropyLoss()
optimizer_sgd=torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=0.97)
scheduler_sgd = ReduceLROnPlateau(optimizer_sgd, 'min', patience=1,factor=0.5)

In [17]:
from tqdm.auto import tqdm
def train_loop(dataloader, model, loss_fn, optimizer):
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    
    train_loss = 0
    total_correct = 0
    total_samples = 0
    model.train()
    for batch, (X, y) in enumerate(tqdm(dataloader, desc="Training", leave=False)):
        X=X.to(device)
        y = y.to(device)
        # forward pass
        y_pred = model(X)
        # Calculate and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss+=loss.item()
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
        total_correct+=(y_pred_class==y).sum().item()
        total_samples+=len(y)

        
    train_loss = train_loss / len(dataloader)
    train_acc = total_correct / total_samples
    
    return train_acc,train_loss


def val_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    
    val_loss=0
    total_correct = 0
    total_samples = 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.inference_mode():
        for batch,(X, y) in enumerate(tqdm(dataloader, desc="Validation", leave=False)):
            X=X.to(device)
            y = y.to(device)
            test_pred_logits = model(X)
            loss=loss_fn(test_pred_logits,y)
            val_loss+=loss.item()
            #Calculate and accumulate Accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            total_correct += (test_pred_labels == y).sum().item()
            total_samples += len(y) # Add the number of samples in this batch
            
    # Adjust metrics to get average loss and accuracy per batch 
    val_loss = val_loss / len(dataloader)
    val_acc = total_correct / total_samples
   
    return val_acc,val_loss

In [ ]:
for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train_acc,train_loss=train_loop(dataloader_train, model, loss_fn, optimizer_sgd)
    val_acc,val_loss=val_loop(dataloader_val, model, loss_fn)
    print(f"Train Error: \n Accuracy: {train_acc*100:.3f}%, Avg loss: {train_loss:.4f} \n")
    print(f"Validation Error: \n Accuracy: {val_acc*100:.3f} %, Avg loss: {val_loss:.4f}\n ")
    current_lr = optimizer_sgd.param_groups[0]['lr']
    print(f"End of Epoch {i+1}, Current LR: {current_lr}")
    scheduler_sgd.step(val_loss)
print("Training Complete !")

Epoch 1
-------------------------------


Training:   0%|          | 0/394 [00:00<?, ?it/s]

Validation:   0%|          | 0/44 [00:00<?, ?it/s]

Train Error: 
 Accuracy: 64.037%, Avg loss: 0.6241 

Validation Error: 
 Accuracy: 78.143 %, Avg loss: 0.5169
 
End of Epoch 1, Current LR: 1e-05
Epoch 2
-------------------------------


Training:   0%|          | 0/394 [00:00<?, ?it/s]

...

Train Error: 
 Accuracy: 81.395%, Avg loss: 0.4412 

Validation Error: 
 Accuracy: 85.000 %, Avg loss: 0.3912
 
End of Epoch 3, Current LR: 1e-05
Epoch 4
-------------------------------


Training:   0%|          | 0/394 [00:00<?, ?it/s]

Validation:   0%|          | 0/44 [00:00<?, ?it/s]

Train Error: 
 Accuracy: 82.594%, Avg loss: 0.4064 

Validation Error: 
 Accuracy: 87.357 %, Avg loss: 0.3564
 
End of Epoch 4, Current LR: 1e-05
Epoch 5
-------------------------------


Training:   0%|          | 0/394 [00:00<?, ?it/s]

In [ ]:
predictions=[]
with torch.inference_mode():
    for X_batch_tuple in tqdm(dataloader_test,desc="Predicting"):
        X_batch=X_batch_tuple[0].to(device)
        output=model(X_batch)
        _,preds=torch.max(output,1)
        predictions.append(preds.cpu())
        final_predictions = torch.cat(predictions)
submission = pd.DataFrame({
    "id": test_dataset.img_names,
    "label": final_predictions.numpy() # Convert tensor to numpy array for pandas
})

In [ ]:
submission.to_csv('submission.csv', index=False)